# Routing
Routing refers to the intelligent selection of the most suitable retrieval path or reasoning logic based on the user's query content, enabling more efficient answer retrieval.

This dynamic selection process is especially important in scenarios with multiple data sources, multiple retrievers, or multiple tasks, as it can significantly improve system performance and accuracy.

## Types of Routing in RAG:
### Logical Routing
### Semantic Routing

![Routing](../resources/RAG/3_RAG_Routing.png)

In [4]:
CHROMA_PATH = "chroma_rag3"

%load_ext autoreload
%autoreload 2

from dbUtils import DBUtils
dbUtils = DBUtils()
dbUtils.clear_database(CHROMA_PATH+"_2023")
dbUtils.clear_database(CHROMA_PATH+"_2024")

In [ ]:
from pdfImportService import pdf_content_into_documents, format_docs

pdf_content_into_documents("../data/US_Forecast_May_2023.pdf", CHROMA_PATH+"_2023")


pdf_content_into_documents("../data/US_Forecast_May_2024.pdf", CHROMA_PATH+"_2024")

USER_AGENT environment variable not set, consider setting it to identify your requests.
/Users/easonyin/Project/github/AIJourney/RAG/dbUtils.py:39: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/Users/easonyin/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


find chuck ../data/US_Forecast_May_2023.pdf:0
find chuck ../data/US_Forecast_May_2023.pdf:1
find chuck ../data/US_Forecast_May_2023.pdf:2
find chuck ../data/US_Forecast_May_2023.pdf:3
find chuck ../data/US_Forecast_May_2023.pdf:4
find chuck ../data/US_Forecast_May_2023.pdf:5
find chuck ../data/US_Forecast_May_2023.pdf:6
find chuck ../data/US_Forecast_May_2023.pdf:7
find chuck ../data/US_Forecast_May_2023.pdf:8
find chuck ../data/US_Forecast_May_2023.pdf:9
find chuck ../data/US_Forecast_May_2023.pdf:10
find chuck ../data/US_Forecast_May_2023.pdf:11
find chuck ../data/US_Forecast_May_2023.pdf:12
find chuck ../data/US_Forecast_May_2023.pdf:13
find chuck ../data/US_Forecast_May_2023.pdf:14
find chuck ../data/US_Forecast_May_2023.pdf:15
find chuck ../data/US_Forecast_May_2023.pdf:16
find chuck ../data/US_Forecast_May_2023.pdf:17
find chuck ../data/US_Forecast_May_2023.pdf:18
find chuck ../data/US_Forecast_May_2023.pdf:18
find chuck ../data/US_Forecast_May_2023.pdf:19
find chuck ../data/US_F

[Document(metadata={'source': '../data/US_Forecast_May_2024.pdf', 'page': 0, 'id': '../data/US_Forecast_May_2024.pdf:0:0'}, page_content="University of Michigan, Ann Arbor \nDepartment of Economics \n611 Tappan Avenue \nAnn Arbor, MI 48109-1120 \n \nlsa.umich.edu/econ/rsqe \n734-764-2567 \n \n \n \n \n \n \n \n \n \n \n \nFor Release: 05/17/2024 \n“The Michigan Model” \nGabriel M. Ehrlich, Director  \nGeorge A. Fulton & Saul H. Hymans \nDirectors Emeriti \n \n \nThe U.S. Economic Outlook for 2024–2025 \n \nJacob T. Burton, Gabriel M. Ehrlich, Kyle W. Henson, Daniil Manaenkov, and Yinuo Zhang \nUniversity of Michigan \nExecutive Summary\nLong Story Short \nWe project the U.S. economy to maintain healthy \nmomentum throughout our forecast window, with a \nmodest deceleration in the near term. We expect real \nGDP growth to average about 2.0 percent annualized in \nthe second half of 2024, while the unemployment rate \ncontinues inching up to 4.0 percent. Meaningful  \nprogress towards in

## Logical Routing
#### Logical Routing uses explicit rules or classification models to direct queries into different paths.
#### For example, it can categorize queries to route them to different databases, knowledge bases, or APIs.

### Suitable for:
#### Structured questions
#### Scenarios with clearly defined data sources

![Logic Routing](../resources/RAG/3_RAG_Routing_Sematic.png)

In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

DEEPSEEK_API_KEY = os.getenv('DEEPSEEK_API_KEY')

In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_deepseek import ChatDeepSeek

# Initialize Deepseek LLM

llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [8]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["2023", "2024"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

# LLM with function call 
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt 
system = """You are an expert at routing a user question to the appropriate data source.

Based on the question is referring to, route it to the relevant data source."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
router = prompt | structured_llm

In [9]:
question = """ What economic trends are in 2023? """


router.invoke({"question": question})

RouteQuery(datasource='2023')

In [10]:
from langchain_core.runnables import RunnableLambda

def choose_route(result):
    if "2023" in result.datasource.lower():
        ### Logic here 
        return CHROMA_PATH+"_2023"
    elif "2024" in result.datasource.lower():
        ### Logic here 
        return CHROMA_PATH+"_2024"
    else:
        ### Logic here 
        return "NONE"
    




In [13]:
def retrieve_with_queries(db_path):

    print("db_path", db_path)
   
    retriever = dbUtils.get_chroma(db_path).as_retriever(search_kwargs={"k": 5})
    return retriever

In [14]:
full_chain = router | RunnableLambda(choose_route) | retrieve_with_queries 

full_chain.invoke({"question": question})

db_path chroma_rag3_2023


[Document(metadata={'id': '../data/US_Forecast_May_2023.pdf:18:1', 'page': 18, 'source': '../data/US_Forecast_May_2023.pdf'}, page_content='• We project consumption growth to stall in 2023H2 and \n2024H1, driven by tightening credit, high interest rates, \nand a gradual return to pre-pandemic patterns of \nconsumption.  \n• As the economy goes through a soft patch, we expect \nlimited investment contribution to growth. By 2024H2, \nwith the Fed easing policy, investment rebounds to \nadd about 1.0 percentage point to GDP growth.'),
 Document(metadata={'id': '../data/US_Forecast_May_2023.pdf:11:0', 'page': 11, 'source': '../data/US_Forecast_May_2023.pdf'}, page_content="12 \nbeen introduced yet. T he new rules of the House of Representatives under Speaker McCarthy's \nleadership preclude packages of spending bills  coming up for a vote at once, necessitating separate \nHouse votes on all twelve appropriation bills. The White House released its fiscal 2024 budget proposal \nin March. It 

# Semantic Routing
Dynamic selection of optimal paths based on vectorized semantic similarity. For example, matching the most suitable prompt or retrieval tool through embedding techniques. Suitable for fuzzy queries or scenarios requiring flexible responses."

In [18]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# Two prompts
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

dbUtils = DBUtils()

embedding_model = dbUtils.get_embedding_function()
# Embed prompts
prompt_templates = [physics_template, math_template]
prompt_embeddings = embedding_model.embed_documents(prompt_templates)

# Route question to prompt 
def prompt_router(input):
    # Embed question
    query_embedding = embedding_model.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt 
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)

In [19]:
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm
    | StrOutputParser()
)

print(chain.invoke("What's a black hole"))

Using PHYSICS
A black hole is a region in space where gravity is so strong that nothing—not even light—can escape its pull. This happens because a massive amount of matter is compressed into an extremely small volume, creating a gravitational field so intense that it warps spacetime.

Key features of a black hole:
1. **Event Horizon**: The boundary around a black hole beyond which nothing can escape. It’s the "point of no return."
2. **Singularity**: At the center, matter is crushed into an infinitely dense point.
3. **Types**: Black holes can be stellar (formed from collapsing stars), supermassive (found at the centers of galaxies), or intermediate.

Black holes are fascinating because they challenge our understanding of physics, especially where quantum mechanics and general relativity intersect. If you have more questions, feel free to ask!
